In [ ]:
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Configuration pour afficher les images dans le notebook
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Import des classes et fonctions depuis src
from src import (
    SERIES_DIR, TRAIN_CSV, TRAIN_LOCALIZERS_CSV, OUTPUT_DIR,
    print_config
)
from src.bricks import EDA, Preprocessor
from src import show_middle_slices

# Afficher la configuration
print("="*60)
print("Configuration de l'environnement")
print("="*60)
print_config()

# Exploration des Données DICOM

Ce notebook utilise la classe **EDA** pour analyser les données locales de manière systématique.

> **Note importante** : Ce notebook utilise un **échantillon de 20 séries DICOM** pour démonstration locale.
> 
> Le projet original sur Kaggle a analysé **4000+ séries** avec la même architecture.
> Cette version permet de tester le code et visualiser la structure des données localement.

In [ ]:
# Charger les données
df_train = pd.read_csv(TRAIN_CSV)
df_localizers = pd.read_csv(TRAIN_LOCALIZERS_CSV)

print(f"Données chargées:")
print(f"  - Séries totales: {len(df_train)}")
print(f"  - Localisateurs (avec anévrismes): {len(df_localizers)}")

In [ ]:
# Créer l'analyseur EDA
eda = EDA(df_train, df_localizers, SERIES_DIR)

print(f"Analyseur EDA créé")
print(eda)

In [ ]:
# Analyser les modalités
modality_counts = eda.analyze_modalities()

In [ ]:
# Analyser la distribution des anévrismes
aneurysm_counts = eda.analyze_aneurysm_distribution()

In [ ]:
# Analyser les positions anatomiques
position_counts = eda.analyze_positions()

In [ ]:
# Visualiser les distributions
eda.plot_aneurysm_distribution()

In [ ]:
# Détecter les séries défectueuses (optionnel - peut être long)
# Commentez cette cellule si vous avez beaucoup de séries

# defective_series = eda.detect_defective_series()
print("Détection des séries défectueuses commentée (peut être long)")
print("Décommentez la cellule pour l'exécuter sur vos 20 séries locales")

In [ ]:
# Analyser le nombre de slices
slice_stats = eda.analyze_slice_counts()

In [ ]:
# Rapport complet
print("\n" + "="*60)
print("RAPPORT COMPLET")
print("="*60)
eda.generate_report()

## Visualisation par Modalité

Utilisons la méthode `visualize_modalities()` de la classe EDA pour afficher des exemples d'images IRM des 4 modalités disponibles.

In [ ]:
# Créer un preprocessor pour la visualisation
preprocessor = Preprocessor(target_spacing=(0.4, 0.4, 0.4))

# Visualiser 1 exemple de chaque modalité
# Cela affichera automatiquement des images CTA, MRA, MRI T2, et MRI T1post
eda.visualize_modalities(preprocessor, max_per_modality=1)

## Visualiser Plus d'Exemples (Optionnel)

Si vous souhaitez voir plus d'exemples par modalité, augmentez le paramètre `max_per_modality`.

In [ ]:
# Décommentez pour voir 2 exemples de chaque modalité
# eda.visualize_modalities(preprocessor, max_per_modality=2)

## Conclusion

Ce notebook a utilisé les composants modulaires de `src/` :

### Classe EDA
- Analyse des modalités (CTA, MRA, MRI T2, MRI T1post)
- Distribution des anévrismes
- Positions anatomiques
- Détection de séries défectueuses
- Statistiques sur les slices
- **Visualisation des images IRM par modalité**

### Classe Preprocessor
- Chargement DICOM
- Resampling à espacement cible
- Cropping du fond
- Normalisation

### Fonctions de Visualisation
- `show_middle_slices()` pour visualiser les coupes 3D (axiale, coronale, sagittale)

### Prochaines étapes

Consultez le notebook [02_dataset_creation.ipynb](02_dataset_creation.ipynb) pour créer un dataset d'entraînement à partir de ces données.